In [5]:
'''Imports'''
import json
import pandas as pd

In [2]:
'''Pre-Processing'''
def load_reddit_comment_data(data_directory):

    comments_data = [] # list object that will store the loaded Reddit comments

    # we first open the file that includes our dataset
    with open(data_directory, 'r', encoding='utf-8') as f:
        # iterate the file, reading it line by line
        for line in f:
            # load the data petraining to a line into a json object in memory
            data = json.loads(line)

            # append the comment
            comments_data.append(data['body'])

    # the method returns all the loaded Reddit comments
    return comments_data

In [3]:
data_dir_comments = r"C:\Users\gungo\OneDrive\Desktop\stocks_comments.ndjson"
data_dir_sub = r"C:\Users\gungo\OneDrive\Desktop\stocks_submissions.ndjson"
reddit_data = load_reddit_comment_data(data_dir_comments)
print("Successfully loaded Reddit comments! Our dataset includes %d Reddit comments!" %len(reddit_data))


Successfully loaded Reddit comments! Our dataset includes 689866 Reddit comments!


In [18]:
df = pd.read_json(data_dir_comments, lines=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 689866 entries, 0 to 689865
Data columns (total 52 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   all_awardings                    689866 non-null  object 
 1   archived                         689866 non-null  bool   
 2   associated_award                 0 non-null       float64
 3   author                           689866 non-null  object 
 4   author_created_utc               603083 non-null  float64
 5   author_flair_background_color    79536 non-null   object 
 6   author_flair_css_class           0 non-null       float64
 7   author_flair_richtext            610330 non-null  object 
 8   author_flair_template_id         0 non-null       float64
 9   author_flair_text                0 non-null       float64
 10  author_flair_text_color          79536 non-null   object 
 11  author_flair_type                610330 non-null  object 
 12  au

In [19]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 689866 entries, 0 to 689865
Data columns (total 52 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   all_awardings                    689866 non-null  object 
 1   archived                         689866 non-null  bool   
 2   associated_award                 0 non-null       float64
 3   author                           689866 non-null  object 
 4   author_created_utc               603083 non-null  float64
 5   author_flair_background_color    79536 non-null   object 
 6   author_flair_css_class           0 non-null       float64
 7   author_flair_richtext            610330 non-null  object 
 8   author_flair_template_id         0 non-null       float64
 9   author_flair_text                0 non-null       float64
 10  author_flair_text_color          79536 non-null   object 
 11  author_flair_type                610330 non-null  object 
 12  au

In [17]:
df = pd.read_json(data_dir_comments, lines=True)
df.info()
df_reduced = df['body']
df_reduced = df.drop(df.columns[[0, 1, 2, 3, 4, 5, 6, 7]], axis=1)
print(df_reduced.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 689866 entries, 0 to 689865
Data columns (total 52 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   all_awardings                    689866 non-null  object 
 1   archived                         689866 non-null  bool   
 2   associated_award                 0 non-null       float64
 3   author                           689866 non-null  object 
 4   author_created_utc               603083 non-null  float64
 5   author_flair_background_color    79536 non-null   object 
 6   author_flair_css_class           0 non-null       float64
 7   author_flair_richtext            610330 non-null  object 
 8   author_flair_template_id         0 non-null       float64
 9   author_flair_text                0 non-null       float64
 10  author_flair_text_color          79536 non-null   object 
 11  author_flair_type                610330 non-null  object 
 12  au